In [22]:
from google.colab import files
files.upload()

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c titanic

Saving kaggle.json to kaggle (1).json
train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [24]:
train= pd.read_csv("train.csv")
test= pd.read_csv("test.csv")
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


カテゴリ変数を数値的変数へ変換 <br>
male : 0<br>
female : 1<br>
S : 0<br>
C : 1<br>
Q : 2<br>

In [0]:
train= pd.read_csv("train.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
test= pd.read_csv("test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

AgeとEmbarkedの欠損値を平均値で埋める

In [0]:
train["Age"].fillna(train.Age.mean(), inplace=True) 
train["Embarked"].fillna(train.Embarked.mean(), inplace=True)

Name　Ticket　Cabin　が処理できていないため、処理する

In [0]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False) 

In [28]:
pd.crosstab(train['Title'], train['Sex'])

Sex,0,1
Title,,
Capt,1,0
Col,2,0
Countess,0,1
Don,1,0
Dr,6,1
Jonkheer,1,0
Lady,0,1
Major,2,0
Master,40,0


Nameを1~5に分割し、欠損値は0とする

In [0]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
del train['Name']

In [0]:
train_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 

train['Title'] = train['Title'].map(train_mapping) 
train['Title'] = train['Title'].fillna(0)

In [31]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,0,22.0,1,0,A/5 21171,7.2500,NaN,0.0,1
1,2,1,1,1,38.0,1,0,PC 17599,71.2833,C85,1.0,3
2,3,1,3,1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0,2
3,4,1,1,1,35.0,1,0,113803,53.1000,C123,0.0,3
4,5,0,3,0,35.0,0,0,373450,8.0500,NaN,0.0,1


Ticketでは、数字と文字が混ざっているため、すべて文字に変換する。

In [0]:
train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x))

In [33]:
train['Ticket_Lett']

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket_Lett, Length: 891, dtype: object

In [41]:
train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
train['Ticket_Lett']

0      A
1      P
2      S
3      1
4      3
      ..
886    2
887    1
888    W
889    1
890    3
Name: Ticket_Lett, Length: 891, dtype: object

In [0]:
train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), 
                                train['Ticket_Lett'], 
                                np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')) 

In [43]:
train['Ticket_Lett']

0      A
1      P
2      S
3      1
4      3
      ..
886    2
887    1
888    0
889    1
890    3
Name: Ticket_Lett, Length: 891, dtype: object

In [45]:
train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
train['Ticket_Len']

0       9
1       8
2      16
3       6
4       6
       ..
886     6
887     6
888    10
889     6
890     6
Name: Ticket_Len, Length: 891, dtype: int64

In [0]:
del train['Ticket'] 
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)

In [47]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title,Ticket_Lett,Ticket_Len
0,1,0,3,0,22.0,1,0,7.2500,NaN,0.0,1,3,9
1,2,1,1,1,38.0,1,0,71.2833,C85,1.0,3,0,8
2,3,1,3,1,26.0,0,0,7.9250,NaN,0.0,2,3,16
3,4,1,1,1,35.0,1,0,53.1000,C123,0.0,3,1,6
4,5,0,3,0,35.0,0,0,8.0500,NaN,0.0,1,3,6


Ticketの頭文字が　1 2 3 S P C A であるものはTrue でそれ以外のものは０にした。 <br>
Ticketの文字数も新しい特徴量に加えた<br>

In [0]:
train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
del train['Cabin'] 
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)

In [49]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Ticket_Lett,Ticket_Len,Cabin_Lett
0,1,0,3,0,22.0,1,0,7.2500,0.0,1,3,9,0
1,2,1,1,1,38.0,1,0,71.2833,1.0,3,0,8,1
2,3,1,3,1,26.0,0,0,7.9250,0.0,2,3,16,0
3,4,1,1,1,35.0,1,0,53.1000,0.0,3,1,6,1
4,5,0,3,0,35.0,0,0,8.0500,0.0,1,3,6,0


In [0]:
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1

train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

In [53]:
train_data = train.values
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Ticket_Lett,Ticket_Len,Cabin_Lett,FamilySize,IsAlone
0,1,0,3,0,22.0,1,0,7.2500,0.0,1,3,9,0,2,0
1,2,1,1,1,38.0,1,0,71.2833,1.0,3,0,8,1,2,0
2,3,1,3,1,26.0,0,0,7.9250,0.0,2,3,16,0,1,1
3,4,1,1,1,35.0,1,0,53.1000,0.0,3,1,6,1,2,0
4,5,0,3,0,35.0,0,0,8.0500,0.0,1,3,6,0,1,1


In [0]:
xs = train_data[:, 2:] # Pclass以降の変数
y  = train_data[:, 1]  # 正解データ

In [0]:
test["Age"].fillna(train.Age.mean(), inplace=True)
test["Fare"].fillna(train.Fare.mean(), inplace=True)

In [0]:
combine = [test]
for test in combine:
    test['Salutation'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
for test in combine:
    test['Salutation'] = test['Salutation'].replace(['Lady', 'Countess','Capt', 'Col',\
         'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    test['Salutation'] = test['Salutation'].replace('Mlle', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Ms', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Mme', 'Mrs')
    del test['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for test in combine:
    test['Salutation'] = test['Salutation'].map(Salutation_mapping)
    test['Salutation'] = test['Salutation'].fillna(0)

for test in combine:
        test['Ticket_Lett'] = test['Ticket'].apply(lambda x: str(x)[0])
        test['Ticket_Lett'] = test['Ticket_Lett'].apply(lambda x: str(x))
        test['Ticket_Lett'] = np.where((test['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), test['Ticket_Lett'],
                                   np.where((test['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0', '0'))
        test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
        del test['Ticket']
test['Ticket_Lett']=test['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 

for test in combine:
        test['Cabin_Lett'] = test['Cabin'].apply(lambda x: str(x)[0])
        test['Cabin_Lett'] = test['Cabin_Lett'].apply(lambda x: str(x))
        test['Cabin_Lett'] = np.where((test['Cabin_Lett']).isin(['T', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']),test['Cabin_Lett'],
                                   np.where((test['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0','0'))        
        del test['Cabin']
test['Cabin_Lett']=test['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1).replace("G",1) 

test["FamilySize"] = train["SibSp"] + train["Parch"] + 1

for test in combine:
    test['IsAlone'] = 0
    test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1

In [57]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett,FamilySize,IsAlone
0,892,3,0,34.5,0,0,7.8292,2,1,3,6,0,2,0
1,893,3,1,47.0,1,0,7.0000,0,3,3,6,0,2,0
2,894,2,0,62.0,0,0,9.6875,2,1,2,6,0,1,1
3,895,3,0,27.0,0,0,8.6625,0,1,3,6,0,2,0
4,896,3,1,22.0,1,1,12.2875,0,3,3,7,0,1,1


In [0]:
test_data = test.values
xs_test = test_data[:, 1:]

In [0]:
from sklearn.ensemble import RandomForestClassifier


random_forest=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=51, n_jobs=4,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
random_forest.fit(xs, y)
Y_pred = random_forest.predict(xs_test)

import csv
with open("predict_result_data2.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test_data[:,0].astype(int), Y_pred.astype(int)):
        writer.writerow([pid, survived])

In [65]:
!kaggle competitions submit -c titanic -f predict_result_data2.csv -m "3%"

100% 2.77k/2.77k [00:08<00:00, 333B/s]
Successfully submitted to Titanic: Machine Learning from Disaster